# Repository Pattern

* Enables decoupling model layer from the data layer

## Applying the DIP to Data Access

* Recall that we're buliding up a 3-layered architecture (presentation layer -> business layer -> data layer)
* Our goal is to keep layers separate - each layer should only depend on the layer below it.
* However, our domain model should have *no* dependencies.
* Infrastructure concerns shouldn't slow our ability to make changes to our domain model
* As a result, let's move from our 3-layered architecture to an onion architecture: presentation layer -> domain model <- database layer.

## Reminder: Our Model

* An `OrderLine` is stored to a `Batch` as an allocation.

### The "Normal" ORM Way: Model Depends on ORM

* Rather than creating custom SQL queries, the team are likely using some framework to generate queries based on model objects.
* These frameworks are called *object-relational mappers* (ORMs).
* They map objects and domain models to databases and relational algebra.
* ORMs give us *persistence ignorance* - domain model doesn't need to know how data is loaded or persisted.

In [6]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

Base = declarative_base()

class Order(Base):
    __tablename__ = "orders"
    id = Column(Integer, primary_key=True)
    
class OrderLine(Base):
    __tablename__ = "orderline"
    id = Column(Integer, primary_key=True)
    sku = Column(String(250))
    quantity = Column(Integer)
    order_id = Column(Integer, ForeignKey('order.id'))
    order = relationship(Order)

* Model classes above depend on ORM classes. We want this to be the other way around

### Inverting the Dependency: ORM Depends on Model

* We can invert this relationship by using [classical mapping](https://docs.sqlalchemy.org/en/14/orm/mapping_styles.html#imperative-a-k-a-classical-mappings).
* See [the reference code](https://github.com/cosmicpython/code/blob/chapter_02_repository/orm.py)
* The `start_mappers` function will associate our model with the database if we call the function. If we never call it, the domain model operates completely independently of the `database` without any issues.

## Introducing the Repository Pattern

* Abstraction for persistent storage
* Hides details about data access by pretending all data is in memory

### The Repository in the Abstract

* The simplest case has just two methods `add()` and `get()`
* Sticking this simplicity stops us from coupling domain model to the database

In [7]:
from abc import ABC, abstractmethod
from typing import Any


class AbstractRepository(ABC):
    
    @abstractmethod
    def add(self, val: Any):
        pass
    
    @abstractmethod
    def get(self, reference) -> Any:
        pass

* Here we use ABCs, but many python devs prefer not to use them since they often end up unmaintained.
* Many pythonistas prefer [PEP 544](https://www.python.org/dev/peps/pep-0544/), which gives typing without inheritance. I like [this tutorial](https://dev.to/meseta/factories-abstract-base-classes-and-python-s-new-protocols-structural-subtyping-20bm)

### What is the Trade-Off?

* Pro: Abstraction over storage layer, which we control.
* Pro: Building a fake repository for testing is trivial.
* Con: Need to add code to our repository class each time we add a new domain object that we want to retrieve.

## What is a Port and What is an Adapter in Python?

* *Port* - interface between our app and what we want to abstract away
* *Adapter* - The implementation behind that interface
* In the above, `AbstractRepository` is the port, and any concretions would be adapters.

## Wrap-Up

* Pros:
    * Simple interface between persistent storage and our domain model
    * Easy to make a fake repo for unit testing since things are fully-decoupled
    * Writing the model before thinging about persistence helps us focus on the actual problem
    * Database schema is simple as we have complete control over how we map our objects to tables
* Cons:
    * An ORM already does some decoupling.
    * Maintaining ORM mappings requires extra code
    * May confuse new devs